In [1]:
#load Text file
#filepath = '/kaggle/input/shakespeare/shakespeare.txt'
filepath = '/kaggle/input/orwell1984/Orwell1984.txt'
with open(filepath) as f:
    document_text = f.read()


In [2]:
#check data
print(document_text[:80])

﻿
Chapter 1

t was a bright cold day in April, and the clocks were strik- ing th


In [3]:
#Encode characters
from tensorflow import keras

text_vec_layer = keras.layers.TextVectorization(split="character", standardize="lower")
text_vec_layer.adapt([document_text])
encoded = text_vec_layer([document_text])[0]

2025-07-19 04:15:18.155680: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1752898518.358560      36 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1752898518.420554      36 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
I0000 00:00:1752898530.980158      36 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 13942 MB memory:  -> device: 0, name: Tesla T4, pci bus id: 0000:00:04.0, compute capability: 7.5
I0000 00:00:1752898530.980859      36 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:1 with 13942 MB memory:  -> device: 1, name: Tesla T4, pci bus id: 0000:00:05.0, compute capability:

In [4]:
#Reduce encoding by 2 as we don't need 0 and 1 reserved by TextVectorization
encoded = encoded - 2 
n_tokens = text_vec_layer.vocabulary_size() - 2 
dataset_size = len(encoded) 
dataset_size

587776

In [5]:
n_tokens

57

In [6]:
#function to prepare dataset
import tensorflow as tf

def to_dataset(sequence, length, shuffle=False, seed=None, batch_size=32):
    ds = tf.data.Dataset.from_tensor_slices(sequence)
    ds = ds.window(length + 1, shift=1, drop_remainder=True)
    ds = ds.flat_map(lambda window_ds: window_ds.batch(length + 1))
    if shuffle:
        ds = ds.shuffle(buffer_size=100_000, seed=seed)
    ds = ds.batch(batch_size)
    return ds.map(lambda window: (window[:, :-1], window[:, 1:])).prefetch(1)

In [7]:
#create test, validation and testing datasets
tf.random.set_seed(42)

length = 100
train_set = to_dataset(encoded[:1_000_000], length=length, shuffle=True, seed=42)
valid_set = to_dataset(encoded[1_000_000:1_060_000], length=length)
test_set = to_dataset(encoded[1_060_000:], length=length)

In [8]:
#create model
model = tf.keras.Sequential([
    keras.layers.Embedding(input_dim=n_tokens, output_dim=16),
    keras.layers.GRU(128, return_sequences=True),
    keras.layers.Dense(n_tokens, activation="softmax")])

In [10]:
#compile model
model.compile(loss="sparse_categorical_crossentropy", 
              optimizer="nadam", 
              metrics=["accuracy"])

In [11]:
#set callbacks
model_ckpt = keras.callbacks.ModelCheckpoint("my_document_model.keras", 
                                                monitor="val_accuracy", 
                                                save_best_only=True)

In [12]:
#train model
history = model.fit(train_set, 
                    validation_data=valid_set, 
                    epochs=5, 
                    callbacks=[model_ckpt])

Epoch 1/5


I0000 00:00:1752898644.207730     101 cuda_dnn.cc:529] Loaded cuDNN version 90300


  18364/Unknown 163s 8ms/step - accuracy: 0.5332 - loss: 1.5821

/usr/local/lib/python3.11/dist-packages/keras/src/trainers/epoch_iterator.py:151: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self._interrupted_warning()


18365/18365 ━━━━━━━━━━━━━━━━━━━━ 163s 8ms/step - accuracy: 0.5332 - loss: 1.5821
Epoch 2/5


/usr/local/lib/python3.11/dist-packages/keras/src/callbacks/model_checkpoint.py:209: UserWarning: Can save best model only with val_accuracy available, skipping.
  self._save_model(epoch=epoch, batch=None, logs=logs)


18365/18365 ━━━━━━━━━━━━━━━━━━━━ 161s 8ms/step - accuracy: 0.6201 - loss: 1.2506
Epoch 3/5
18365/18365 ━━━━━━━━━━━━━━━━━━━━ 158s 8ms/step - accuracy: 0.6285 - loss: 1.2185
Epoch 4/5
18365/18365 ━━━━━━━━━━━━━━━━━━━━ 158s 8ms/step - accuracy: 0.6325 - loss: 1.2016
Epoch 5/5
18365/18365 ━━━━━━━━━━━━━━━━━━━━ 162s 8ms/step - accuracy: 0.6347 - loss: 1.1917


In [14]:
#save model
model = model.save('my_model.keras')

In [32]:
# Save Tokenizer i.e. Vocabulary
import pickle
with open('tokenizer.pickle', 'wb') as handle:
    pickle.dump(text_vec_layer, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [23]:
#load model
model = keras.models.load_model("/kaggle/working/my_model.keras")

In [24]:
#data processing for input text going to model at end
document_model = keras.Sequential([text_vec_layer,
                                         keras.layers.Lambda(lambda X: X - 2),
                                         model
                                        ])

In [25]:
#new input text

#input_text = "To be or not to b"
input_text = "To obey the law is "
y_proba = document_model.predict(tf.constant([input_text]) )[0, -1]

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 246ms/step


In [26]:
#predict new character
y_pred = tf.argmax(y_proba)
text_vec_layer.get_vocabulary()[y_pred + 2]

't'

In [27]:
log_probas = tf.math.log([[0.5, 0.4, 0.1]])
tf.random.set_seed(42)
tf.random.categorical(log_probas, num_samples=8)

<tf.Tensor: shape=(1, 8), dtype=int64, numpy=array([[0, 0, 1, 1, 1, 0, 0, 0]])>

In [28]:
#function to generate next character
def next_char(text, temperature=1):
    y_proba = document_model.predict(tf.constant([text]))[0, -1:]
    rescaled_logits = tf.math.log(y_proba) / temperature
    char_id = tf.random.categorical(rescaled_logits, num_samples=1)[0, 0]
    return text_vec_layer.get_vocabulary()[char_id + 2]

In [29]:
#function to generate several characters
def extend_text(text, n_chars=50, temperature=1):
    for _ in range(n_chars):
        text += next_char(text, temperature)
    return text

In [30]:
tf.random.set_seed(42)

In [31]:
#testing
print(extend_text(input_text, temperature=0.01))

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 78ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━

In [ ]:
print(extend_text(input_text, temperature=1))

In [ ]:
print(extend_text(input_text, temperature=1.5))

In [ ]:
print(extend_text(input_text, temperature=10))

In [64]:
!pip install streamlit
!npm install localtunnel

⠙⠹⠸⠼⠴
up to date, audited 23 packages in 704ms
⠴
⠴3 packages are looking for funding
⠴  run `npm fund` for details
⠴
2 high severity vulnerabilities

To address all issues (including breaking changes), run:
  npm audit fix --force

Run `npm audit` for details.
⠴

In [21]:
!npm audit fix --force

npm warn using --force Recommended protections disabled.
⠙npm error code ENOLOCK
⠙npm error audit This command requires an existing lockfile.
⠙npm error audit Try creating one first with: npm i --package-lock-only
⠙npm error audit Original error: loadVirtual requires existing shrinkwrap file
⠙npm error A complete log of this run can be found in: /root/.npm/_logs/2025-07-19T04_55_59_482Z-debug-0.log
⠙

In [1]:
%%writefile app.py

import streamlit as st
import pickle
from tensorflow import keras

with open('/kaggle/working/tokenizer.pickle', 'rb') as f:
    tokenizer = pickle.load(f)

model = tf.keras.models.load_model('/kaggle/working/my_model.keras')

document_model = keras.Sequential([tokenizer,
                                   keras.layers.Lambda(lambda X: X - 2),
                                   model])

st.title('GRU Text generator')
st.write('Using Orwell 1984')

user_input = st.chat_input(placeholder="User")
if user_input:
    pred = extend_text(input_text, temperature=1)
    st.write(pred)

Writing app.py
